<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><li></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

**Импортируем библиотеки.**

In [11]:
import pandas as pd 
import numpy as np

In [8]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [10]:
import spacy
import re 
import nltk
from nltk.corpus import stopwords 
from sklearn.pipeline import Pipeline

**Записываем в переменную csv-файл.**

In [12]:
url = 'https://github.com/MEfse/project/blob/main/Project/%D0%9C%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5%20%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B4%D0%BB%D1%8F%20%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2/toxic_comments.csv?raw=true'

In [13]:
data = pd.read_csv(url, index_col=[0])

In [4]:
data = data[:90000]

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [16]:
def clear_text(text):  
    return ' '.join(re.sub(r'[^a-zA-Z]', ' ', text).split())

In [8]:
print(data['text'].head(3))

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
Name: text, dtype: object


In [21]:
spacy_en = spacy.load('en_core_web_sm')

In [10]:
data['lemm_text'] = data['text'].apply(lambda x : " ".join(token.lemma_ for token in spacy_en(x)))

In [11]:
for x in range(data['lemm_text'].shape[0]):
    data['lemm_text'][x] = clear_text(data['lemm_text'][x])

/tmp/ipykernel_303/2282150346.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['lemm_text'][x] = clear_text(data['lemm_text'][x])


In [12]:
print(data['lemm_text'].head(5))

0    Explanation why the edit make under my usernam...
1    D aww he match this background colour I be see...
2    hey man I be really not try to edit war it be ...
3    More I can not make any real suggestion on imp...
4    you sir be my hero any chance you remember wha...
Name: lemm_text, dtype: object


In [13]:
stop_words = set(stopwords.words('english')) 

Данные подготовлены.

## Обучение

In [14]:
X = data['lemm_text']
y = data['toxic']

In [15]:
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=.25, random_state=42)

In [16]:
print(f'Размер обучающей выборки: {X_train.shape[0]} процент от общей {(X_train.shape[0] / data.shape[0]) * 100}%')
print(f'Размер тестовой выборки: {X_test.shape[0]} процент от общей {(X_test.shape[0] / data.shape[0]) * 100}%')

Размер обучающей выборки: 67500 процент от общей 75.0%
Размер тестовой выборки: 22500 процент от общей 25.0%


In [17]:
count_vect = CountVectorizer() 
bow = count_vect.fit_transform(X_train) 

In [18]:
count_tf_idf = TfidfVectorizer() 

In [19]:
tf_idf = count_tf_idf.fit_transform(X_train) 

In [20]:
parameters = {'C' : np.logspace(-3,5,9),
              'solver': ['liblinear'],
              'penalty': ['l1','l2'],
              'max_iter' : [30, 80]}

In [21]:
model_lr = LogisticRegression()

In [22]:
grid_model = GridSearchCV(model_lr,
                          parameters, 
                          scoring='f1', 
                          n_jobs=-1, 
                          cv=5)

In [23]:
result = grid_model.fit(tf_idf, y_train)
f1_score_grid = abs(result.best_score_).round(2)

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("L

In [24]:
print(f'{result.best_params_}')
print(f"F1 мера: {f1_score_grid}")

{'C': 10.0, 'max_iter': 80, 'penalty': 'l1', 'solver': 'liblinear'}
F1 мера: 0.77


In [25]:
pipe_lr = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('model', LogisticRegression())])

In [26]:
model = pipe_lr.fit(X_train, y_train)
prediction = model.predict(X_test)
f1_score_pipe = round(f1_score(y_test, prediction),2)

In [27]:
print(f'f1_score: {f1_score_pipe}')

f1_score: 0.72


In [28]:
score_f1 = pd.DataFrame({'GridSearchCV' : [f1_score_grid],
                         'Pipeline' : [f1_score_pipe]}, index=['F1 score'])

In [29]:
display(score_f1)

,GridSearchCV,Pipeline
F1 score,0.77,0.72


## Вариант 2 

In [24]:
import torch
from tqdm import notebook

In [29]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [26]:
display(device)

device(type='cpu')

In [27]:
%%time
batch_size = 2 
embeddings = [] 
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids[batch_size*i:batch_size*(i+1)]).cuda() 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()

        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings

features = np.concatenate(embeddings) 

NameError: name 'input_ids' is not defined

## Выводы

В данном проекте подготовлены данные: полностью очищен от шума и подготовлен текст. Произведено обучение моделей и получена f1 метрика не ниже 0.75. 

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны